In [1]:
import sys
import numpy as np
from copy import deepcopy

sys.path.append("./src/")
from pdb_numpy import Coor, Model
from pdb_numpy.format import mmcif, pdb

In [2]:

test = Coor('src/pdb_numpy/tests/input/2ol9.cif')
print(test.len)
test.write("test_raw.pdb", check_file_out=False)

56


In [3]:
test.transformation

{1: {'chains': ['A', 'B'],
  'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 1.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 1.0, 0.0],
   [2.0, 1.0, 0.0, 0.0, 0.9401502764],
   [2.0, 0.0, 1.0, 0.0, 4.7875628934],
   [2.0, 0.0, 0.0, 1.0, 0.0]]}}

In [4]:
test.apply_transformation()
test.write("test_trans.pdb", check_file_out=False)
print(test.len)
#assert test.len == 112

112


In [5]:

# Here there is only identity matrix
# as symmetry, so no change is expected
test.add_symmetry()
print(test.len)
test.write("test_sym.pdb", check_file_out=False)

No symmetry matrix found.


112


In [6]:
test.copy_box(3, 3, 3)
assert test.len == 3024
test.compute_chains_CA()
test.remove_overlap_chain()
assert test.len == 3024

test.write("test.pdb", check_file_out=False)

In [7]:
test.write('tmp.pdb', check_file_out=False)

In [8]:
test_2 = Coor('tmp.pdb')
test_2.write('tmp2.gro', check_file_out=False)

In [10]:
test.add_symmetry()
test.apply_transformation(index=1)
test.compute_chains_CA()
test.write('tmp.pdb', check_file_out=False)

No symmetry matrix found.


In [11]:
test.symmetry

''

In [12]:
test.models

In [13]:
print(test.transformation)

{1: {'chains': ['A', 'B'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0], [2.0, 1.0, 0.0, 0.0, 0.9401502764], [2.0, 0.0, 1.0, 0.0, 4.7875628934], [2.0, 0.0, 0.0, 1.0, 0.0]]}}


In [14]:
def parse_transformation(text):

    transformation_dict = {}

    for line in text.split('\n'):
        if line[11:23] == "BIOMOLECULE:":
            biomol = int(line[24:])
            transformation_dict[biomol] = {'chains': [], 'matrix': []}
        elif line[34:41] == "CHAINS:":
            transformation_dict[biomol]['chains'] += line[42:].split()
        elif line.startswith('REMARK 350   BIOMT'):
            transformation_dict[biomol]['matrix'] += [[float(x) for x in line[19:].split()]]
        #print(line, line[34:])
    print(transformation_dict)


parse_transformation(test.transformation)

AttributeError: 'dict' object has no attribute 'split'

In [34]:
test.models.append(deepcopy(test.models[0]))
test.models.append(deepcopy(test.models[0]))

In [4]:
text = \
"""REMARK 350   BIOMT1   2 -1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   2  0.000000  1.000000  0.000000        7.05000            
REMARK 350   BIOMT3   2  0.000000  0.000000 -1.000000        0.00000"""

rot_mat = []

for line in text.split("\n"):
    rot_mat.append([float(x) for x in line.split()[4:]])

rot_mat = np.array(rot_mat, dtype=object)
print(rot_mat)

text = \
"""REMARK 350   BIOMT1   3 -1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   3  0.000000  1.000000  0.000000       -2.35000            
REMARK 350   BIOMT3   3  0.000000  0.000000 -1.000000        0.00000"""

rot_mat_2 = []

for line in text.split("\n"):
    rot_mat_2.append([float(x) for x in line.split()[4:]])

rot_mat_2 = np.array(rot_mat_2, dtype=object)
print(rot_mat_2)

[[-1.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 7.05]
 [0.0 0.0 -1.0 0.0]]
[[-1.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 -2.35]
 [0.0 0.0 -1.0 0.0]]


In [21]:
rot_mat[:,:3]

array([[-1.0, 0.0, 0.0],
       [0.0, 1.0, 0.0],
       [0.0, 0.0, -1.0]], dtype=object)

In [36]:
test.models[1].xyz = np.dot(test.models[0].xyz, rot_mat[:,:3]) + rot_mat[:,3]
test.models[2].xyz = np.dot(test.models[0].xyz, rot_mat_2[:,:3]) + rot_mat_2[:,3]

In [37]:
test.merge_model()
test.write_pdb('tmp.pdb', check_file_out=False)

In [25]:
pwd

'/home/murail/Documents/Code/pdb_numpy'